<a href="https://colab.research.google.com/github/mmontana83/Laboratorio-An-lisis-de-Datos-de-Ventas-Mensuales/blob/main/Laboratorio_An%C3%A1lisis_de_Datos_de_Ventas_Mensuales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from datetime import datetime, timedelta

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Genero una lista con las rutas de todos los archivos .CSV

ruta_de_acceso = '/content/drive/MyDrive/Upso/Segundo año/Segundo Cuatrimestre/Big Data/LABORATORIO 1'

archivos = os.listdir(ruta_de_acceso)
ruta_completa = []
for archivo in archivos:
  if archivo.endswith('.csv'): #Asegurarse de que se consideren archivos csv
    ruta_completa.append(os.path.join(ruta_de_acceso,archivo)) # Ruta de acceso completa

In [ ]:
df_ventas = []

#Genero el DataFrame concatenando todos los archivos
df_ventas = pd.concat(map(pd.read_csv, ruta_completa), ignore_index=True)

#Transformo todos los valores no numericos en Nan en las columnas Cantidad Pedida y Precio Unitario
df_ventas['Cantidad Pedida'] = pd.to_numeric(df_ventas['Cantidad Pedida'], errors='coerce')  #El errors= 'coerce' transforma los valores no numericos a NaN'
df_ventas['Precio Unitario'] = pd.to_numeric(df_ventas['Precio Unitario'], errors='coerce')

# Elimino todos los Nan y me quedo solo con los valores numericos
df_ventas = df_ventas.loc[~df_ventas['Cantidad Pedida'].isna()]
df_ventas = df_ventas.loc[~df_ventas['Precio Unitario'].isna()]

df_ventas = df_ventas.dropna()

# Transformo todos los valores numericos y fechas
df_ventas['Cantidad Pedida'] = df_ventas['Cantidad Pedida'].astype(int)
df_ventas['Precio Unitario'] = df_ventas['Precio Unitario'].astype(float).round(decimals = 2)

#Estas dos listas las voy a usar para que los campos [nombreDia] y [Mes] sean del tipo Categorical para poder mantenerlos ordenados
dias_ordenados = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
meses_ordenados = ["January", "February", "March", "April", "May", "June", "July", "August", "September", "October", "November", "December"]

#Cambio el tipo de dato de la columna [Fecha de Pedido] a datetime en una nueva columna[Fecha Convertida]
df_ventas['Fecha Convertida'] = pd.to_datetime(df_ventas['Fecha de Pedido'])
df_ventas = df_ventas.drop("Fecha de Pedido", axis = 1)

#Agrego las columnas [nombreDia] [Dia] [Mes] [Hora]
df_ventas['nombreDia'] = df_ventas['Fecha Convertida'].dt.day_name()
df_ventas['nombreDia'] = pd.Categorical(df_ventas['nombreDia'], dias_ordenados)
df_ventas['nombreDia'] = df_ventas['nombreDia'].map({"Monday":"Lunes", "Tuesday":"Martes", "Wednesday":"Miércoles", "Thursday":"Jueves",
                                                     "Friday":"Viernes", "Saturday":"Sábado", "Sunday":"Domingo"})

#Esta Columna representa el Número de Día obtenido de la columna [Fecha Convertida]
df_ventas['Dia'] = df_ventas['Fecha Convertida'].dt.day

df_ventas['Mes'] = df_ventas['Fecha Convertida'].dt.month_name()
df_ventas['Mes'] = pd.Categorical(df_ventas['Mes'], meses_ordenados)
df_ventas['Mes'] = df_ventas['Mes'].map({"January":"Enero", "February":"Febrero", "March":"Marzo", "April":"Abril", "May":"Mayo",
                                                         "June":"Junio", "July":"Julio", "August":"Agosto", "September":"Septiembre", "October":"Octubre",
                                                         "November":"Noviembre", "December":"Diciembre"})

df_ventas['Hora'] = df_ventas['Fecha Convertida'].dt.hour

#Agrego la Columna Precio Total
df_ventas['Precio Total'] = df_ventas['Cantidad Pedida']*df_ventas['Precio Unitario']
df_ventas['Precio Total'] = df_ventas['Precio Total'].astype(float).round(decimals = 2)

#Separo la columnas [Direccion de Envio] en las columnas [Ciudad][Estado][CP]
#Esto lo hago así para poder distinguir las dos ciudades Portland que se llaman igual pero son de distintos estados.
df_ventas['Ciudad'] = df_ventas['Dirección de Envio'].apply(lambda x: x.split(',')[1]) + ', ' + df_ventas['Dirección de Envio'].apply(lambda x: x.split(',')[2].split(' ')[1])
df_ventas['Estado'] = df_ventas['Dirección de Envio'].apply(lambda x: x.split(',')[2].split(' ')[1])

df_ventas['Ciudad'] = df_ventas['Ciudad'].astype(str)
df_ventas['Estado'] = df_ventas['Estado'].astype(str)

#Ordeno columnas
df_ventas = df_ventas[['ID de Pedido', 'Producto', 'Cantidad Pedida', 'Precio Unitario', 'Precio Total',
                           'Fecha Convertida', 'nombreDia','Dia', 'Mes','Hora','Dirección de Envio','Ciudad','Estado']]

#Ordeno las Filas por fecha
df_ventas = df_ventas.sort_values('Fecha Convertida')

#Elimino las fechas 2020 para hacer solamente análisis sobre el 2019
df_ventas.drop(df_ventas[(df_ventas['Fecha Convertida'] >= '2020-01-01')].index, inplace=True)

#Reindexo
df_ventas.reset_index(drop = True, inplace = True) #averiguar que hace drop

df_ventas

,ID de Pedido,Producto,Cantidad Pedida,Precio Unitario,Precio Total,Fecha Convertida,nombreDia,Dia,Mes,Hora,Dirección de Envio,Ciudad,Estado
0,147268,Wired Headphones,1,11.99,11.99,2019-01-01 03:07:00,Martes,1,Enero,3,"9 Lake St, New York City, NY 10001","New York City, NY",NY
1,148041,USB-C Charging Cable,1,11.95,11.95,2019-01-01 03:40:00,Martes,1,Enero,3,"760 Church St, San Francisco, CA 94016","San Francisco, CA",CA
2,149343,Apple Airpods Headphones,1,150.00,150.00,2019-01-01 04:56:00,Martes,1,Enero,4,"735 5th St, New York City, NY 10001","New York City, NY",NY
3,149964,AAA Batteries (4-pack),1,2.99,2.99,2019-01-01 05:53:00,Martes,1,Enero,5,"75 Jackson St, Dallas, TX 75001","Dallas, TX",TX
4,149350,USB-C Charging Cable,2,11.95,23.90,2019-01-01 06:03:00,Martes,1,Enero,6,"943 2nd St, Atlanta, GA 30301","Atlanta, GA",GA
...,...,...,...,...,...,...,...,...,...,...,...,...,...
185911,297481,AA Batteries (4-pack),1,3.84,3.84,2019-12-31 23:42:00,Martes,31,Diciembre,23,"82 Hill St, Dallas, TX 75001","Dallas, TX",TX
185912,298406,AAA Batteries (4-pack),2,2.99,5.98,2019-12-31 23:42:00,Martes,31,Diciembre,23,"30 Elm St, San Francisco, CA 94016","San Francisco, CA",CA
185913,317048,AAA Batteries (4-pack),2,2.99,5.98,2019-12-31 23:45:00,Martes,31,Diciembre,23,"743 Adams St, San Francisco, CA 94016","San Francisco, CA",CA
185914,309773,AAA Batteries (4-pack),1,2.99,2.99,2019-12-31 23:52:00,Martes,31,Diciembre,23,"60 Hickory St, Los Angeles, CA 90001","Los Angeles, CA",CA


#1. Comportamiento de las ventas en los distintos meses:
##● ¿Cómo variaron las ventas a lo largo de los diferentes meses?
##● ¿Hubo algún mes que se destacó en términos de ventas?

In [ ]:
#Genero una tabla temporal con dos columnas: Mes y Cantidad Pedida
df_cantidadPedidaPorMes = df_ventas.groupby(['Mes'])['Cantidad Pedida'].sum(list).reset_index()
promedioPedidos = df_cantidadPedidaPorMes['Cantidad Pedida'].mean()

#Genero el gráfico de barras
figCantidadPedidaPorMes = px.bar(df_cantidadPedidaPorMes, x="Mes", y="Cantidad Pedida", color="Cantidad Pedida", barmode="group", text= "Cantidad Pedida")
figPorcentajePedidosPorMes = px.pie(df_cantidadPedidaPorMes, values="Cantidad Pedida", names="Mes")
figCantidadPedidaPorMes.add_hline(y=promedioPedidos,annotation_text="Promedio de Unidades Vendidas")

figCantidadPedidaPorMes.update_layout(
    title="Unidades vendidas por mes",
    title_x = 0.5,
    xaxis=dict(title="Meses"),
    yaxis=dict(title="Unidades vendidas", side="left"))

figPorcentajePedidosPorMes .update_layout(
    title="Porcentaje de Ventas por mes",
    title_x = 0.5)

figPorcentajePedidosPorMes.update_traces(textposition='inside', textinfo='percent+label')

figCantidadPedidaPorMes.show()
figPorcentajePedidosPorMes .show()

In [ ]:

#Genero una tabla temporal con dos columnas: Mes y Cantidad Pedida

df_cantidadPedidaPorMesPorcentual = df_ventas.groupby(['Mes'])['Cantidad Pedida'].sum(list).reset_index()

# Obtengo el Porcentaje de cada mes respecto de la venta anual
ventasAnual = df_ventas['Cantidad Pedida'].sum()
for Mes in df_cantidadPedidaPorMesPorcentual:
  df_cantidadPedidaPorMesPorcentual['Porcentaje'] =  df_cantidadPedidaPorMes['Cantidad Pedida'] * 100 / ventasAnual

## FORMULA PORCENTUAL DE LOS SEMESTRES

porcentajePrimerSemestre = df_cantidadPedidaPorMesPorcentual['Porcentaje'][0:6].sum()
porcentajeSegundoSemestre =df_cantidadPedidaPorMesPorcentual['Porcentaje'][6:12].sum()

## FORMULA PORCENTUAL DE LOS TRIMESTRES
primerTrimestre = df_cantidadPedidaPorMesPorcentual['Porcentaje'][0:3].sum()
segundoTrimestre = df_cantidadPedidaPorMesPorcentual['Porcentaje'][3:6].sum()
tercerTrimestre = df_cantidadPedidaPorMesPorcentual['Porcentaje'][6:9].sum()
cuartoTrimestre = df_cantidadPedidaPorMesPorcentual['Porcentaje'][9:12].sum()

figSemestres = go.Figure() ## Declaro figura de torta para los Semestres
## Carga de datos a la figura de Semestre
trace_pieSemestre = go.Pie(labels=['Primer Semestre', 'Segundo Semestre'], values= [porcentajePrimerSemestre, porcentajeSegundoSemestre], legendgroup='1', sort=False, textinfo='percent+label')

figTrimestres = go.Figure() ## Declaro figura de torta para los Trimestres
## Carga de datos a la figura de los Trimestres
trace_pieTrimestre = go.Pie(labels = ['Primer Trimestre', 'Segundo Trimestre', 'Tercer Trimestre', 'Cuarto Trimestre'],
                            values= [primerTrimestre, segundoTrimestre, tercerTrimestre, cuartoTrimestre],
                            legendgroup='2',
                            sort=False,
                            textinfo='percent+label')

#muestro ambas figuras
fig = make_subplots(rows=1, cols=2, subplot_titles=('Ventas Semestrales', 'Ventas trimestrales'),specs=[[{"type": "pie"}, {"type": "pie"}]])
fig.add_trace(trace_pieSemestre, row=1, col=1)
fig.add_trace(trace_pieTrimestre, row=1, col=2)

fig.update_layout(
    title_text="Ventas Anuales",
    title_x= 0.45,
    showlegend=True
)

fig.show()



##● ¿Cuál es el ingreso total generado por mes?

In [ ]:
#Genero una tabla temporal con dos columnas: Mes y Precio Total
#agregar porcentaje sobre el total
df_ingresoTotalPorMes = df_ventas.groupby(['Mes'])['Precio Total'].sum(list).round(decimals = 2).reset_index()
promedioIngresos = df_ingresoTotalPorMes['Precio Total'].mean().round(decimals = 2)

#Genero el gráfico de barras
figIngresoTotalPorMes = px.bar(df_ingresoTotalPorMes, x="Mes", y="Precio Total", color="Precio Total", barmode="group", text= "Precio Total")
figIngresoTotalPorMes.add_hline(y=promedioIngresos,annotation_text="Promedio de Ingresos")

figIngresoTotalPorMes.update_layout(
    title="Ingreso Total generado por mes",
    title_x = 0.5,
    xaxis=dict(title="Meses"),
    yaxis=dict(title="Precio Total", side="left"),
    yaxis_tickprefix = '$', yaxis_tickformat = "${:,.0f}",#Esto es para mostrar el símbolo $)
    )

#Agrego el Símbolo $
figIngresoTotalPorMes.update_traces(text=df_ingresoTotalPorMes['Precio Total'].map("$ {:,.2f}".format))

figIngresoTotalPorMes.show()
promedioIngresos

2873613.81

#2. Optimización de la publicidad y patrón de ventas por hora:
##● ¿Cuál es el momento ideal del día para presentar la publicidad y aumentar la probabilidad de compra?

In [ ]:
#Genero una tabla temporal con dos columnas: Hora y Cantidad Pedida
df_ventasPorHora= df_ventas.groupby(['Hora'])['Cantidad Pedida'].sum(list).reset_index()

#Genero el gráfico de barras
figVentasPorHora = px.line(df_ventasPorHora, x="Hora", y="Cantidad Pedida", markers = True, text = "Cantidad Pedida")

figVentasPorHora.update_layout(
    title="Cantidad de Ventas por Hora",
    title_x = 0.5,
    xaxis=dict(title="Horas", tickvals=df_ventasPorHora['Hora']),
    yaxis=dict(title="Cantidad Pedida", side="left")
    )

#Esto es para agregar los valores sobre los puntos
figVentasPorHora.update_traces(textposition="bottom left")

figVentasPorHora.show()

##● ¿Cómo cambian los patrones de ventas por hora a lo largo del año?

In [ ]:
#El siguiente dataframe me devuelve las cantidades vendidas agrupadas por Mes y por Hora
df_patronesVentaAño = df_ventas.groupby(['Mes','Hora'])['Cantidad Pedida'].sum(list).reset_index()

fig = go.Figure()

#Genero las trazas
for Mes, group in df_patronesVentaAño.groupby('Mes'):
    fig.add_trace(go.Scatter(x=group["Hora"], y=group["Cantidad Pedida"], name=Mes))

# Configuro los ejes y los títulos
fig.update_layout(
    title="Unidades vendidas por hora por Mes",
    title_x = 0.5,
    xaxis=dict(title="Hora", tickvals=df_patronesVentaAño.Hora),  # Para que en el eje x se muestren todos los valores uso tickvals=Horas
    yaxis=dict(title="Unidades vendidas", side="left")
    )

fig.show()

In [ ]:
df_ventasPorHora = df_ventas.groupby(['Fecha Convertida','Hora'])['Cantidad Pedida'].sum(list).reset_index()
df_ventasPorHora['Fecha Convertida'] = df_ventasPorHora['Fecha Convertida'].dt.dayofyear
df_ventasPorHora = df_ventasPorHora.groupby(['Fecha Convertida','Hora'])['Cantidad Pedida'].sum(list).reset_index()

fig = go.Figure(data=go.Heatmap(
    x=df_ventasPorHora['Fecha Convertida'],
    y=df_ventasPorHora['Hora'],
    z=df_ventasPorHora['Cantidad Pedida'],
    colorscale='Viridis',
    colorbar=dict(title='Cantidad Pedida'))
    )

# Personalizo el diseño del heatmap
fig.update_layout(
    title='Heatmap de Cantidad Pedida por Hora y Dias',
    title_x = 0.5,
    xaxis=dict(title='Dias del Año'),
    yaxis=dict(title='Horas del dia',tickvals=df_ventasPorHora.Hora),
    plot_bgcolor='#440154',  xaxis_zeroline=False, yaxis_zeroline=False,
    xaxis_showgrid=False, yaxis_showgrid=False
)

# Muestro el heatmap interactivo
fig.show()

##● ¿Hay modificaciones en los patrones de ventas durante las horas de mayor actividad en los distintos meses?

In [ ]:
#El siguiente dataframe me devuelve las cantidades vendidas agrupadas por Mes y por Hora
df_patronesVentaHorasMayorActividad = df_ventas.groupby(['Hora','Mes'])['Cantidad Pedida'].sum(list).reset_index()

#Filtro solamente la Hora con más ventas
df_patronesVentaHorasMayorActividad = df_patronesVentaHorasMayorActividad[(df_patronesVentaHorasMayorActividad.Hora == 19) | (df_patronesVentaHorasMayorActividad.Hora == 12)]

fig = go.Figure()

#Genero las trazas
for Hora, group in df_patronesVentaHorasMayorActividad.groupby('Hora'):
  fig.add_trace(go.Scatter(x=group["Mes"], y=group["Cantidad Pedida"], name=Hora,
                           line = dict(width= 2, shape= 'spline')))

# Configuro los ejes y los títulos
fig.update_layout(
    title="Unidades vendidas en las Horas de Mayor Actividad por Mes",
    title_x = 0.5,
    xaxis=dict(title="Meses"),
    yaxis=dict(title="Unidades vendidas", side="left")
    )

fig.show()

#3. Distribución de ventas por ubicación:
##● ¿En qué ciudades se han registrado las mayores ventas?

In [ ]:
df_mayoresVentasPorCiudad = df_ventas.groupby(['Ciudad'])['Cantidad Pedida'].sum(list).reset_index()
df_mayoresVentasPorCiudad = df_mayoresVentasPorCiudad.sort_values('Cantidad Pedida')

#Genero el gráfico de barras y de los mapas
figCiudad = px.bar(df_mayoresVentasPorCiudad, x="Ciudad", y="Cantidad Pedida", color="Cantidad Pedida", barmode="group", text= "Cantidad Pedida")

figCiudad.update_layout(
    title="Cantidad de Ventas por Ciudad",
    title_x = 0.5,
    xaxis=dict(title="Ciudades"),
    yaxis=dict(title="Unidades vendidas", side="left"))

figCiudad.show()


In [ ]:
df_mayoresVentasPorCiudadPorcentual = df_mayoresVentasPorCiudad = df_ventas.groupby(['Ciudad'])['Cantidad Pedida'].sum(list).reset_index()

## en VentasAnual almaceno el total de ventas en el año
ventasAnual = df_ventas['Cantidad Pedida'].sum()
ventasAnual

# Ciclo que me carga el porcentaje que representa las ventas de cada mes
for ciudad in df_mayoresVentasPorCiudad:
  df_mayoresVentasPorCiudadPorcentual['Porcentaje'] =  df_mayoresVentasPorCiudad['Cantidad Pedida'] * 100 / ventasAnual


fig = go.Figure()

df_mayoresVentasPorCiudadPorcentual = df_mayoresVentasPorCiudadPorcentual.sort_values('Porcentaje')

Ciudad = df_mayoresVentasPorCiudadPorcentual['Ciudad']
Cantidad= df_mayoresVentasPorCiudadPorcentual['Cantidad Pedida']
Porcentaje = df_mayoresVentasPorCiudadPorcentual['Porcentaje']

fig = px.bar(df_mayoresVentasPorCiudadPorcentual, x='Ciudad', y='Cantidad Pedida', color="Cantidad Pedida", title='Porcentaje de Ventas por Ciudad', labels={'ventas': 'Porcentaje de Ventas'}, text= "Porcentaje",)
#fig.add_trace(go.bar(df_cantidadPedidaPorMesPromedio, x='Mes', y='Promedio', color="Cantidad Pedida", title='Promedio de Ventas por Mes', labels={'ventas': 'Promedio de Ventas'}, text= "Promedio",)


fig.update_layout(
    xaxis_title='Ciudad',
    yaxis_title='Porcentaje de Ventas',
    title ='Porcentajes de Ventas por Ciudad',
    title_x = 0.5)

fig.update_traces(text= df_mayoresVentasPorCiudadPorcentual['Porcentaje'].map("{:,.2f}%".format))

fig.show()

##● ¿Cómo se comparan las ventas en diferentes estados o regiones?

In [ ]:
df_mayoresVentasPorEstado = df_ventas.groupby(['Estado'])['Cantidad Pedida'].sum(list).reset_index()

df_mayoresVentasPorEstado = df_mayoresVentasPorEstado.sort_values('Cantidad Pedida')

figBarEstado = px.bar(df_mayoresVentasPorEstado, x="Estado", y="Cantidad Pedida", color="Cantidad Pedida", barmode="group", text= "Cantidad Pedida")

figBarEstado.update_layout(
    title="Cantidad de Ventas por Estado",
    title_x = 0.5,
    xaxis=dict(title="Estados"),
    yaxis=dict(title="Unidades vendidas", side="left"))

figMapEstado = px.choropleth(df_mayoresVentasPorEstado, locations = 'Estado', locationmode='USA-states', scope='usa', color='Cantidad Pedida', color_continuous_scale='RdYlGn')

figMapEstado.update_layout(
    title='Cantidad de Ventas por Estado',
    title_x = 0.5)

figBarEstado.show()
figMapEstado.show()

##● ¿Existe variación en las ventas por estado a lo largo de los meses?

In [ ]:
#El siguiente dataframe me devuelve las cantidades vendidas agrupadas por Mes y por Hora
df_variacionVentasPorEstadoPorMes = df_ventas.groupby(['Mes','Ciudad'])['Cantidad Pedida'].sum(list).reset_index()

fig = px.bar(df_variacionVentasPorEstadoPorMes, x="Mes", y="Cantidad Pedida",
             color="Ciudad",
             barmode = 'group')

# Configuro los ejes y los títulos
fig.update_layout(
    title="Variación de las Unidades Vendidas en los Estados por Mes",
    title_x = 0.5,
    xaxis=dict(title="Meses"),  # Para que en el eje x se muestren todos los valores uso tickvals=Horas
    yaxis=dict(title="Unidades vendidas", side="left")
    )

fig.show()

In [ ]:
fig_line = px.line(df_variacionVentasPorEstadoPorMes, x="Mes", y="Cantidad Pedida",
             color="Ciudad",
             markers= True
             )


# Configuro los ejes y los títulos
fig_line.update_layout(
    title="Variación de las Unidades Vendidas en los Estados por Mes",
    title_x = 0.5,
    xaxis=dict(title="Meses"),  # Para que en el eje x se muestren todos los valores uso tickvals=Horas
    yaxis=dict(title="Unidades vendidas", side="left")
    )

fig_line.show()

#4. Análisis del producto más vendido:
##● ¿Cuál es el producto más vendido en general?


In [ ]:
df_productoMasVendido = df_ventas.groupby(['Producto'])['Cantidad Pedida'].sum(list).reset_index()

df_productoMasVendido = df_productoMasVendido.sort_values('Cantidad Pedida')

#Genero el gráfico de barras
fig = px.bar(df_productoMasVendido, x="Producto", y="Cantidad Pedida", color="Cantidad Pedida", barmode="group", text= "Cantidad Pedida")

fig.update_layout(
    title="Cantidades Vendidas por Producto",
    title_x = 0.5,
    xaxis=dict(title="Producto"),
    yaxis=dict(title="Unidades vendidas", side="left"))

fig.show()

##● ¿y en cada mes?

In [ ]:
#El siguiente dataframe me devuelve las cantidades vendidas agrupadas por Mes y por Hora
df_productoMasVendidoPorMes = df_ventas.groupby(['Mes','Producto'])['Cantidad Pedida'].sum().reset_index()

#Hallar el máximo de cada mes. Tiene que quedar una tabla con 12 filas
df_productoMasVendidoPorMes = df_productoMasVendidoPorMes.groupby('Mes').apply(lambda x: x.loc[x['Cantidad Pedida'].idxmax()]).reset_index(drop= True)

fig = go.Figure()

fig = px.bar(df_productoMasVendidoPorMes, x = 'Mes', y = 'Cantidad Pedida',
             text = 'Cantidad Pedida', color='Producto')

fig.update_layout(
    title="Producto más vendido por Mes",
    title_x = 0.5,
    xaxis=dict(title="Meses"),
    yaxis=dict(title="Unidades vendidas", side="left"))

fig.show()

##● ¿Qué factores crees que han contribuido al éxito de ese producto en particular?

In [ ]:
#Obtengo una tabla con tres columnas [Producto][Precio][Cantidad Pedida == 1]
df_precioProductos = df_ventas.groupby(['Producto'])['Producto', 'Precio Unitario', 'Cantidad Pedida'].apply(lambda x: x.loc[x['Cantidad Pedida'].idxmin()]).reset_index(drop= True)
#Elimino la columna [Cantidad Pedida]
df_precioProductos = df_precioProductos.drop(['Cantidad Pedida'], axis=1)

df_precioProductos = df_precioProductos.sort_values('Precio Unitario')

#Genero el gráfico de barras
fig = px.bar(df_precioProductos, x="Producto", y="Precio Unitario", color="Precio Unitario", barmode="group", text= "Precio Unitario")

fig.update_layout(
    title="Precio de los Productos",
    title_x = 0.5,
    xaxis=dict(title="Producto"),
    yaxis=dict(title="Precio", side="left"),
    yaxis_tickprefix = '$', yaxis_tickformat = "${:,.0f}",#Esto es para mostrar el símbolo $)
)

#Agrego el Símbolo $
fig.update_traces(text=df_precioProductos['Precio Unitario'].map("$ {:,.2f}".format))

fig.show()

<ipython-input-20-451f603586b1>:2: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



#5. Tendencia ventas:
##● ¿Existe alguna tendencia o patrón en las ventas que se repita a lo largo de los días en los diferentes meses?

In [ ]:
#Genero una tabla temporal con dos columnas: Hora y Cantidad Pedida
df_ventasPorDia = df_ventas.groupby(['Dia'])['Cantidad Pedida'].sum(list).reset_index()

#Genero el gráfico de barras
figventasPorDia = px.line(df_ventasPorDia, x="Dia", y="Cantidad Pedida")

figventasPorDia.update_layout(
    title="Ventas por Dia",
    title_x = 0.5,
    xaxis=dict(title="Dias", tickvals=[1,25,50,75,100,125,150,175,200,225,250,275,300,325,350,365]),
    yaxis=dict(title="Cantidad Pedida", side="left")
    )

#Esto es para agregar los valores sobre los puntos
figventasPorDia.update_traces(textposition="bottom left")

figventasPorDia.show()

In [ ]:
df_analisisVentasPorDiasPorMes = df_ventas.groupby(['Mes','nombreDia'])['Cantidad Pedida'].sum(list).reset_index()

fig1analisisVentasPorDiasPorMes = go.Figure()

#Genero las trazas
for Mes, group in df_analisisVentasPorDiasPorMes.groupby('Mes'):
    fig1analisisVentasPorDiasPorMes.add_trace(go.Bar(x=group["nombreDia"],
                             y=group["Cantidad Pedida"],
                             name=Mes))

# Configuro los ejes y los títulos
fig1analisisVentasPorDiasPorMes.update_layout(
    title="Analisis de Ventas por Día en los Distintos Meses",
    title_x = 0.5,
    xaxis=dict(title="Dias", tickvals=group["nombreDia"]),  # Para que en el eje x se muestren todos los valores uso tickvals=Horas
    yaxis=dict(title="Unidades vendidas", side="left")
    )

fig1analisisVentasPorDiasPorMes.show()

In [ ]:
fig2analisisVentasPorDiasPorMes = make_subplots(rows=3, cols=4, subplot_titles=df_analisisVentasPorDiasPorMes['Mes'].unique())

fig2analisisVentasPorDiasPorMes.add_bar(x=df_analisisVentasPorDiasPorMes['nombreDia'][0:7], y=df_analisisVentasPorDiasPorMes['Cantidad Pedida'][0:7], text = df_analisisVentasPorDiasPorMes['Cantidad Pedida'][0:7], showlegend=False, col = 1, row = 1)
fig2analisisVentasPorDiasPorMes.add_bar(x=df_analisisVentasPorDiasPorMes['nombreDia'][7:14], y=df_analisisVentasPorDiasPorMes['Cantidad Pedida'][7:14], text = df_analisisVentasPorDiasPorMes['Cantidad Pedida'][7:14], showlegend=False, col = 2, row = 1)
fig2analisisVentasPorDiasPorMes.add_bar(x=df_analisisVentasPorDiasPorMes['nombreDia'][14:21], y=df_analisisVentasPorDiasPorMes['Cantidad Pedida'][14:21], text = df_analisisVentasPorDiasPorMes['Cantidad Pedida'][14:21], showlegend=False, col = 3, row = 1)
fig2analisisVentasPorDiasPorMes.add_bar(x=df_analisisVentasPorDiasPorMes['nombreDia'][21:28], y=df_analisisVentasPorDiasPorMes['Cantidad Pedida'][21:28], text = df_analisisVentasPorDiasPorMes['Cantidad Pedida'][21:28], showlegend=False, col = 4, row = 1)
fig2analisisVentasPorDiasPorMes.add_bar(x=df_analisisVentasPorDiasPorMes['nombreDia'][28:35], y=df_analisisVentasPorDiasPorMes['Cantidad Pedida'][28:35], text = df_analisisVentasPorDiasPorMes['Cantidad Pedida'][28:35], showlegend=False, col = 1, row = 2)
fig2analisisVentasPorDiasPorMes.add_bar(x=df_analisisVentasPorDiasPorMes['nombreDia'][35:42], y=df_analisisVentasPorDiasPorMes['Cantidad Pedida'][35:42], text = df_analisisVentasPorDiasPorMes['Cantidad Pedida'][35:42], showlegend=False, col = 2, row = 2)
fig2analisisVentasPorDiasPorMes.add_bar(x=df_analisisVentasPorDiasPorMes['nombreDia'][42:49], y=df_analisisVentasPorDiasPorMes['Cantidad Pedida'][42:49], text = df_analisisVentasPorDiasPorMes['Cantidad Pedida'][42:49], showlegend=False, col = 3, row = 2)
fig2analisisVentasPorDiasPorMes.add_bar(x=df_analisisVentasPorDiasPorMes['nombreDia'][49:56], y=df_analisisVentasPorDiasPorMes['Cantidad Pedida'][49:56], text = df_analisisVentasPorDiasPorMes['Cantidad Pedida'][49:56], showlegend=False, col = 4, row = 2)
fig2analisisVentasPorDiasPorMes.add_bar(x=df_analisisVentasPorDiasPorMes['nombreDia'][56:63], y=df_analisisVentasPorDiasPorMes['Cantidad Pedida'][56:63], text = df_analisisVentasPorDiasPorMes['Cantidad Pedida'][56:63], showlegend=False, col = 1, row = 3)
fig2analisisVentasPorDiasPorMes.add_bar(x=df_analisisVentasPorDiasPorMes['nombreDia'][63:70], y=df_analisisVentasPorDiasPorMes['Cantidad Pedida'][63:70], text = df_analisisVentasPorDiasPorMes['Cantidad Pedida'][63:70], showlegend=False, col = 2, row = 3)
fig2analisisVentasPorDiasPorMes.add_bar(x=df_analisisVentasPorDiasPorMes['nombreDia'][70:77], y=df_analisisVentasPorDiasPorMes['Cantidad Pedida'][70:77], text = df_analisisVentasPorDiasPorMes['Cantidad Pedida'][70:77], showlegend=False, col = 3, row = 3)
fig2analisisVentasPorDiasPorMes.add_bar(x=df_analisisVentasPorDiasPorMes['nombreDia'][77:84], y=df_analisisVentasPorDiasPorMes['Cantidad Pedida'][77:84], text = df_analisisVentasPorDiasPorMes['Cantidad Pedida'][77:84], showlegend=False, col = 4, row = 3)

fig2analisisVentasPorDiasPorMes.update_layout(
    title="Análisis por Día por Mes",
    title_x = 0.5)

fig2analisisVentasPorDiasPorMes.show()

##● ¿Cómo varían las ventas a lo largo de los diferentes días de la semana?

In [ ]:
#Genero una tabla temporal con dos columnas: Mes y Cantidad Pedida
df_cantidadVentasDiasSemana = df_ventas.groupby(['nombreDia'])['Cantidad Pedida'].sum(list).reset_index()

#Genero el gráfico de barras

fig = px.bar(df_cantidadVentasDiasSemana, x="nombreDia", y="Cantidad Pedida", color="Cantidad Pedida", barmode="group", text= "Cantidad Pedida")


################# AGREGO GRAFICO DE TORTAS

figTorta = go.Figure()

# Define una escala de colores personalizada (reversa)
color_scale = [ "orange", "green", "yellow", "blue","red","pink","purple"]

dias = df_cantidadVentasDiasSemana['nombreDia']
cantidades = df_cantidadVentasDiasSemana['Cantidad Pedida']

trace_pieTrimestre = go.Pie(labels=dias, values= cantidades, hole=0.3, marker=dict(colors=color_scale))
figTorta.add_trace(trace_pieTrimestre)

figTorta.update_layout(
    title="Cantidad de Ventas por Día en la Semana",
    title_x=0.5,
    coloraxis_showscale=False  # Desactiva la escala de colores en la gráfica de torta
)

####################################################################################

# Define la escala de colores personalizada
color_scale = [
    (0, "red"),  # Color rojo para valores bajos
    (0.5, "yellow"),  # Color amarillo para valores intermedios
    (1, "green")  # Color verde para valores altos
]


fig.update_layout(
    title="Cantidad de Ventas por Dia en la Semana",
    title_x = 0.5,
    yaxis_range=[29000, 31000],
    xaxis=dict(title="Dias de la Semana"),
    yaxis=dict(title="Unidades vendidas", side="left"),
    coloraxis_colorbar=dict(title="Cantidad Pedida"),  # Título de la barra de colores
    coloraxis=dict(colorscale=color_scale)  # Escala de colores personalizada
)

fig.show()
figTorta.show()

##● ¿Se observa alguna diferencia significativa en las ventas entre los días laborables y los fines de semana?

In [ ]:
nombreDiaES = ["Lunes", "Martes", "Miércoles", "Jueves", "Viernes", "Sábado", "Domingo"]

cantidadTotaldeVentas = df_ventas['Precio Total'].sum()

cantidadVentasDiasSemana = 0
for nombreDia in nombreDiaES[:5]:
  cantidadVentasDiasSemana += df_ventas[df_ventas.nombreDia == nombreDia]['Precio Total'].sum()

cantidadVentasFinDeSemana = 0
for nombreDia in nombreDiaES[5:7]:
  cantidadVentasFinDeSemana += df_ventas[df_ventas.nombreDia == nombreDia]['Precio Total'].sum()

filas = [('Semana Laboral', cantidadVentasDiasSemana),
          ('Fin de Semana', cantidadVentasFinDeSemana)]

df_cantidadVentasDiasSemanaDiscriminado = pd.DataFrame(filas, columns=['Periodo', 'Precio Total'])

#Genero el gráfico de barras
fig = px.bar(df_cantidadVentasDiasSemanaDiscriminado, x="Periodo", y="Precio Total", color="Precio Total", barmode="group", text= "Precio Total")

fig.update_layout(
    title="Comparación de Ventas entre Semana Laboral y Fin de Semana",
    title_x = 0.5,
    xaxis=dict(title="Período"),
    yaxis=dict(title="Porcentaje Vendido", side="left"))

#Agrego el Símbolo $
fig.update_traces(text=df_cantidadVentasDiasSemanaDiscriminado['Precio Total'].map("$ {:,.2f}".format))

fig.show()

In [ ]:
nombreDiaES = ["Lunes", "Martes", "Miércoles", "Jueves", "Viernes", "Sábado", "Domingo"]

cantidadTotaldeVentas = df_ventas['Cantidad Pedida'].sum()

cantidadVentasDiasSemana = 0
for nombreDia in nombreDiaES[:5]:
  cantidadVentasDiasSemana += df_ventas[df_ventas.nombreDia == nombreDia]['Cantidad Pedida'].sum()

cantidadVentasFinDeSemana = 0
for nombreDia in nombreDiaES[5:7]:
  cantidadVentasFinDeSemana += df_ventas[df_ventas.nombreDia == nombreDia]['Cantidad Pedida'].sum()


filas = [('Semana Laboral', cantidadVentasDiasSemana*100/5),
          ('Fin de Semana', cantidadVentasFinDeSemana*100/2)]

df_cantidadVentasDiasSemanaDiscriminado = pd.DataFrame(filas, columns=['Periodo', 'Cantidad Pedida'])

#Genero el gráfico de barras
fig = px.bar(df_cantidadVentasDiasSemanaDiscriminado, x="Periodo", y="Cantidad Pedida", color="Cantidad Pedida", barmode="group", text= "Cantidad Pedida")

fig.update_layout(
    title="Promedio de Cantidades Pedidas entre Semana Laboral y Fin de Semana",
    title_x = 0.5,
    yaxis_range=[2900000, 3000000],
    xaxis=dict(title="Período"),
    yaxis=dict(title="Promedio de Pedidos", side="left"))

fig.show()

#6. Impacto de eventos especiales en las ventas:
##● ¿Se ha observado algún aumento o disminución significativa en las ventas en días cercanos a eventos especiales, como días festivos?

##● ¿Qué eventos específicos han tenido un impacto notable en el comportamiento de las ventas y cómo se manifestó ese impacto?

In [ ]:
promedioMayo = df_ventas[(df_ventas['Fecha Convertida'] >= '2019/05/01') & (df_ventas['Fecha Convertida'] < '2019/06/01')]['Cantidad Pedida'].sum()/30
promedioMayo = promedioMayo.round(decimals = 2)
promedioMayo

622.23

In [ ]:
#Genero una tabla con 3 columnas.
df_analisisDiasFestivos = pd.DataFrame(columns = ['Dia Festivo', 'Fecha', 'Cantidad Pedida'])

diasFestivos = {"Día de las madres": datetime.strptime("12-05-2019", "%d-%m-%Y"),
                "Día de la Independencia": datetime.strptime("04-07-2019", "%d-%m-%Y"),
                "Día del Trabajo": datetime.strptime("02-09-2019", "%d-%m-%Y"),
                "Día de los Veteranos": datetime.strptime("11-11-2019", "%d-%m-%Y"),
                "Día de Acción de Gracias": datetime.strptime("23-11-2019", "%d-%m-%Y"),
                "Navidad" : datetime.strptime("24-12-2019", "%d-%m-%Y"),
                "Año Nuevo" :datetime.strptime("31-12-2019", "%d-%m-%Y")}

#por cada dia festivo, sumo la cantidad de pedidos en los 5 días anteriores
for diaFestivo in diasFestivos:
#  print(diaFestivo, diasFestivos[diaFestivo] )
  i=1
  fechaFestiva = diasFestivos[diaFestivo]
  for i in range(6):
    ventas = df_ventas[(df_ventas['Fecha Convertida'] > fechaFestiva - timedelta(1)) & (df_ventas['Fecha Convertida'] < fechaFestiva)]['Cantidad Pedida'].sum()
    fila = [diaFestivo, fechaFestiva , ventas]
    fechaFestiva = fechaFestiva - timedelta(1)
    df_analisisDiasFestivos.loc[len(df_analisisDiasFestivos)] = fila

#Ordeno las Filas por fecha
df_analisisDiasFestivos = df_analisisDiasFestivos.sort_values('Fecha')
df_analisisDiasFestivos = df_analisisDiasFestivos.reset_index(drop = True)

figDiasFestivos = make_subplots(rows=3, cols=3, subplot_titles=df_analisisDiasFestivos['Dia Festivo'].unique())

figDiasFestivos.add_bar(x=df_analisisDiasFestivos['Fecha'][0:6], y=df_analisisDiasFestivos['Cantidad Pedida'][0:6], text = df_analisisDiasFestivos['Cantidad Pedida'][0:6], showlegend=False, col = 1, row = 1)
figDiasFestivos.add_bar(x=df_analisisDiasFestivos['Fecha'][6:12], y=df_analisisDiasFestivos['Cantidad Pedida'][6:12], text = df_analisisDiasFestivos['Cantidad Pedida'][6:12], showlegend=False,col = 2, row = 1)
figDiasFestivos.add_bar(x=df_analisisDiasFestivos['Fecha'][12:18], y=df_analisisDiasFestivos['Cantidad Pedida'][12:18], text = df_analisisDiasFestivos['Cantidad Pedida'][12:18], showlegend=False,col = 3, row = 1)
figDiasFestivos.add_bar(x=df_analisisDiasFestivos['Fecha'][18:24], y=df_analisisDiasFestivos['Cantidad Pedida'][18:24], text = df_analisisDiasFestivos['Cantidad Pedida'][18:24], showlegend=False,col = 1, row = 2)
figDiasFestivos.add_bar(x=df_analisisDiasFestivos['Fecha'][24:30], y=df_analisisDiasFestivos['Cantidad Pedida'][24:30], text = df_analisisDiasFestivos['Cantidad Pedida'][24:30], showlegend=False, col = 2, row = 2)
figDiasFestivos.add_bar(x=df_analisisDiasFestivos['Fecha'][30:36], y=df_analisisDiasFestivos['Cantidad Pedida'][30:36], text = df_analisisDiasFestivos['Cantidad Pedida'][30:36], showlegend=False, col = 3, row = 2)
figDiasFestivos.add_bar(x=df_analisisDiasFestivos['Fecha'][36:42], y=df_analisisDiasFestivos['Cantidad Pedida'][36:42], text = df_analisisDiasFestivos['Cantidad Pedida'][36:42], showlegend=False, col = 1, row = 3)

#calculo el promedio de ventas del mes para comparar con la fecha en cuestion
promedioMayo = df_ventas[(df_ventas['Fecha Convertida'] >= '2019/05/01') & (df_ventas['Fecha Convertida'] < '2019/06/01')]['Cantidad Pedida'].sum()/30
promedioMayo = promedioMayo.round(decimals = 2)

promedioJulio = df_ventas[(df_ventas['Fecha Convertida'] >= '2019/07/01') & (df_ventas['Fecha Convertida'] < '2019/08/01')]['Cantidad Pedida'].sum()/31
promedioJulio = promedioJulio.round(decimals = 2)

promedioAgosto = df_ventas[(df_ventas['Fecha Convertida'] >= '2019/08/01') & (df_ventas['Fecha Convertida'] < '2019/09/01')]['Cantidad Pedida'].sum()/31
promedioAgosto = promedioAgosto.round(decimals = 2)

promedioNoviembre = df_ventas[(df_ventas['Fecha Convertida'] >= '2019/11/01') & (df_ventas['Fecha Convertida'] < '2019/12/01')]['Cantidad Pedida'].sum()/30
promedioNoviembre = promedioNoviembre.round(decimals = 2)

promedioDiciembre = df_ventas[(df_ventas['Fecha Convertida'] >= '2019/12/01') & (df_ventas['Fecha Convertida'] < '2020/01/01')]['Cantidad Pedida'].sum()/31
promedioDiciembre = promedioDiciembre.round(decimals = 2)

figDiasFestivos.add_hline(y=promedioMayo, col = 1, row = 1, annotation_text = 'Promedio de Ventas',)
figDiasFestivos.add_hline(y=promedioJulio, col = 2, row = 1, annotation_text = 'Promedio de Ventas')
figDiasFestivos.add_hline(y=promedioAgosto, col = 3, row = 1, annotation_text = 'Promedio de Ventas')
figDiasFestivos.add_hline(y=promedioNoviembre, col = 1, row = 2, annotation_text = 'Promedio de Ventas')
figDiasFestivos.add_hline(y=promedioNoviembre, col = 2, row = 2, annotation_text = 'Promedio de Ventas')
figDiasFestivos.add_hline(y=promedioDiciembre, col = 3, row = 2, annotation_text = 'Promedio de Ventas')
figDiasFestivos.add_hline(y=promedioDiciembre, col = 1, row = 3, annotation_text = 'Promedio de Ventas')

#Actualizo el rango de los ejes Y para obtener mejor visualización
figDiasFestivos.update_yaxes(range=[500, 700], row=1, col=1)
figDiasFestivos.update_yaxes(range=[450, 600], row=1, col=2)
figDiasFestivos.update_yaxes(range=[350, 500], row=1, col=3)
figDiasFestivos.update_yaxes(range=[550, 800], row=2, col=1)
figDiasFestivos.update_yaxes(range=[550, 800], row=2, col=2)
figDiasFestivos.update_yaxes(range=[800, 1000], row=2, col=3)
figDiasFestivos.update_yaxes(range=[800, 1000], row=3, col=1)

figDiasFestivos.update_layout(
    title="Análisis de Ventas en (5) días cercados a Fechas Festivas",
    title_x = 0.5)

figDiasFestivos.show()